# Chemical-Disease Relation (CDR) Tutorial

In this example, we'll be writing an application to extract *mentions of* **chemical-induced-disease relationships** from Pubmed abstracts, as per the [BioCreative CDR Challenge](http://www.biocreative.org/resources/corpora/biocreative-v-cdr-corpus/).  This tutorial will show off some of the more advanced features of Snorkel, so we'll assume you've followed the Intro tutorial.

Let's start by reloading from the last notebook.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
from snorkel import SnorkelSession

session = SnorkelSession()

In [2]:
from snorkel.models import candidate_subclass

ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])

train = session.query(ChemicalDisease).filter(ChemicalDisease.split == 0).all()
dev = session.query(ChemicalDisease).filter(ChemicalDisease.split == 1).all()
test = session.query(ChemicalDisease).filter(ChemicalDisease.split == 2).all()

print('Training set:\t{0} candidates'.format(len(train)))
print('Dev set:\t{0} candidates'.format(len(dev)))
print('Test set:\t{0} candidates'.format(len(test)))

Training set:	8272 candidates
Dev set:	888 candidates
Test set:	4620 candidates


In [3]:
from snorkel.annotations import load_gold_labels
L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)

# Computing features

For `SparseLogReg`

In [4]:
from snorkel.annotations import FeatureAnnotator
featurizer = FeatureAnnotator()

try:
    F_train = featurizer.load_matrix(session, split=0)
    F_dev = featurizer.load_matrix(session, split=1)
    assert F_train.nonzero() > 0
    assert F_dev.nonzero() > 0
except:
    %time F_train = featurizer.apply(split=0)
    %time F_dev = featurizer.apply_existing(split=1)

Load marginals and dev labels:

In [5]:
from snorkel.annotations import load_marginals
train_marginals = load_marginals(session, F_train, split=0)

In [6]:
from snorkel.annotations import load_gold_labels
L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_dev

<888x1 sparse matrix of type '<type 'numpy.int64'>'
	with 888 stored elements in Compressed Sparse Row format>

# Testing stability of `SparseLogisticRegression` on GPU

In [9]:
from snorkel.learning import SparseLogisticRegression

model_hyperparams = {
    'n_epochs' : 50,
    'rebalance' : 0.5,
    'print_freq' : 5
}

log_reg = SparseLogisticRegression(seed=123, deterministic=True)
log_reg.train(F_train, train_marginals, **model_hyperparams)

(?, ?)
<unknown>
<unknown>
(2, 1)
[SparseLogisticRegression] Training model
[SparseLogisticRegression] n_train=4740  #epochs=50  batch size=256
[SparseLogisticRegression] Epoch 0 (0.25s)	Average loss=0.688807
[SparseLogisticRegression] Epoch 5 (1.57s)	Average loss=0.661567
[SparseLogisticRegression] Epoch 10 (2.95s)	Average loss=0.646704
[SparseLogisticRegression] Epoch 15 (4.49s)	Average loss=0.639022
[SparseLogisticRegression] Epoch 20 (5.93s)	Average loss=0.634158
[SparseLogisticRegression] Epoch 25 (7.61s)	Average loss=0.631077
[SparseLogisticRegression] Epoch 30 (8.94s)	Average loss=0.629019
[SparseLogisticRegression] Epoch 35 (10.70s)	Average loss=0.627127
[SparseLogisticRegression] Epoch 40 (12.16s)	Average loss=0.626166
[SparseLogisticRegression] Epoch 45 (13.61s)	Average loss=0.625724
[SparseLogisticRegression] Epoch 49 (14.69s)	Average loss=0.625085
[SparseLogisticRegression] Training done (14.69s)


In [8]:
print("{0:.5f}".format(log_reg.score(F_dev, L_gold_dev)[-1]))
w1, b1 = log_reg.get_weights()

0.55556


In [10]:
print("{0:.5f}".format(log_reg.score(F_dev, L_gold_dev)[-1]))
w2, b2 = log_reg.get_weights()
print(np.linalg.norm(w1-w2))
print(np.linalg.norm(b1-b2))

0.55556
0.0
0.0


### No seed, det=False
* 0.52891, 0.52878
* 50.2766
* 0.00731753

### Seed=123, det=False
* 0.52066, 0.51962
* 2.69907
* 0.00837359
* t = 19.7 sec


### Seed=123, det=True
* 0.52708, 0.51647
* 2.99769
* 0.00625005
* t = 19.7 sec

# Part V: Training an LSTM extraction model

In the intro tutorial, we automatically featurized the candidates and trained a linear model over these features. Here, we'll train a more complicated model for relation extraction: an LSTM network. You can read more about LSTMs [here](https://en.wikipedia.org/wiki/Long_short-term_memory) or [here](http://colah.github.io/posts/2015-08-Understanding-LSTMs/). An LSTM is a type of recurrent neural network and automatically generates a numerical representation for the candidate based on the sentence text, so no need for featurizing explicitly as in the intro tutorial. LSTMs take longer to train, and Snorkel doesn't currently support hyperparameter searches for them. We'll train a single model here, but feel free to try out other parameter sets. Just make sure to use the development set - and not the test set - for model selection.

**Note: Again, training for more epochs than below will greatly improve performance- try it out!**

In [ ]:
from snorkel.learning import reRNN

train_kwargs = {
    'lr':         0.01,
    'dim':        50,
    'n_epochs':   20,
    'dropout':    0.5,
    'rebalance':  0.25,
    'print_freq': 5,
    'batch_size': 50,
}

lstm = reRNN(seed=1701, n_threads=None)
lstm.train(train, train_marginals, X_dev=dev, Y_dev=L_gold_dev, **train_kwargs)

In [ ]:
lstm.score(test, L_gold_test)